Use this notebook to quickly checkout the wave plot and mel spectogram of downloaded samples.  
Add your cells after the **Explore samples below:** cell. The cells before the **Explore samples below:** cell only need to be run when opening the notebook.  
!!!Changes to this notebook will not be tracked.!!!

## Instructions
display sample metadat with the display_sample_metadata function by passing the sample id.  
Load a sample with the `load_audio_sample` function by passing the sample id as a parameter.  
Pass the returned sample into the show_amplitude_wave_plot function to view the waveplot or the show_mel_spectogram function to view the mel spectogram of the sample

## Example:
```Python
#display metadata
display_sample_metadata(123456789)
#get sample as time series
sample, _ = load_audio_sample(123456789)
'''
New cell >
'''
show_amplitude_wave_plot(sample)
'''
New cell > 
'''
show_mel_spectogram(sample)
```

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import IPython.display as ipd
import os
import json

main_sample_rate = 44100 # 44.1 kHz

samples_metadata = None # dict for species sample info
species_info_df = None # dict with species info

working_dir = os.path.abspath('')

raw_sample_dir = os.path.join(working_dir, 'dataset', 'raw') #path pointing to samples
samples_metadata_file_path = os.path.join(working_dir, 'dataset', 'data_dictionary', 'samples_metadata.json')
species_info_path = os.path.join(working_dir, 'dataset', 'data_dictionary', 'species_info.csv')

In [ ]:
def get_sample_metadata():
    '''
    loads the file that holds information on the downloaded samples and the relevant species
    '''
    with open(samples_metadata_file_path) as f:
        samples_metadata = json.load(f)
    return samples_metadata

def get_species_info():
    '''
    loads the file that holds information on all species
    '''
    species_info_df = pd.read_csv(species_info_path).set_index('species_key')
    return species_info_df

def display_sample_metadata(sample_id):
    '''
    displays the meta data + some additional info on the species in the sample. Takes the sample id as a parameter 
    '''
    if samples_metadata is None:
        raise Exception("sample_meta_data has not been loaded yet. Please run load_sample_meta_data() before this function")
    sample_info = samples_metadata.get(str(sample_id), None)
    if sample_info is None:
        raise Exception("sample with id '{}' was not found in sample_metadata".format(sample_id))
    print('-' * 90)
    print("Sample bgifID: {:>10}".format(sample_info['gbifID']))
    print("url: {}".format(sample_info['recording_link']))
    print("rec time (sec): {}".format(sample_info['recording_time_sec']))
    print("rec date: {}".format(sample_info["date"]))
    print()
    print("decimal latitude: {}".format(sample_info['decimal_latitude']))
    print("decimal longitude: {}".format(sample_info['decimal_longitude']))
    main_species_key = sample_info['forefront_bird_key']
    print()
    print("main species key: {}".format(main_species_key))
    print("main scientific name: {}".format(species_info_df.at[main_species_key, 'scientific_name']))
    print("main common name: {}".format(species_info_df.at[main_species_key, 'common_name']))
    print()
    if sample_info['behavior']:
        print("Noted behavior in this sample:")
        for index, behavior in enumerate(sample_info['behavior']):
            print("\t{} {}".format(str(index)+')', behavior))
    else:
        print("Noted behavior in this sample: None")
    if sample_info['background_birds_keys']:
        print("Background bird's species keys and info:")
        print("\t{:3} {:^10} | {:30} | {:30}".format("", "key", "scientific name", "common_name"))
        print("\t{:^15}┼{:32}┼{:30}".format("-"*15, "-"*32, "-"*30))
        for index, key in enumerate(sample_info['background_birds_keys']):
            print("\t{:3} {:10} | {:30} | {:30}".format(str(index)+')', key, species_info_df.at[key, 'scientific_name'],species_info_df.at[key, 'common_name']))
    else:
        print("Background bird's species keys: None")
    print('-' * 90)

def load_audio_sample(sample_id):
    '''
    loads an audio sample by it's id (file name). Returns the audio and the the sample rate
    '''
    audio_file_path = os.path.join(raw_sample_dir, str(sample_id) + '.mp3')
    if not os.path.isfile(audio_file_path): #check if file exists
        raise Exception("Sample with id '{}' does not exist! Available files can be found in the {} directory".format(sample_id, raw_sample_dir))
    #sample audio at 44.1 khz and get the time series as a numpy array
    time_series, sample_rate = librosa.load(audio_file_path, sr = main_sample_rate)
    #trim empty start and end padding from time series
    time_series, _ = librosa.effects.trim(time_series)

    return time_series, sample_rate

def show_amplitude_wave_plot(sample_time_series):
    '''
    plots the amplitude vs time of the sample 
    '''
    librosa.display.waveplot(sample_time_series, sr=main_sample_rate)

def show_mel_spectogram(sample_time_series):
    '''
    plots the melspectogram of the sample
    '''
    S = librosa.feature.melspectrogram(y=sample_time_series, sr=main_sample_rate, n_mels=128, fmax=10000, center = False)
    plt.figure(figsize=(10, 4))
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time',
                            y_axis='mel', sr=main_sample_rate,
                            fmax=10000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram')
    plt.tight_layout()
    plt.show()

In [ ]:
samples_metadata = get_sample_metadata()
species_info_df = get_species_info()

# Explore samples below: